In [1]:
import json
from IPython.display import display, Markdown

# Define parameters.

PROMPTS = "../data/prompts/nothing/taskvine_nothing_gpt4o_1_analysis.json"

In [2]:
# Analyze a singular prompt.

def analyze(prompt: dict):
    print(prompt["content"])
    display(Markdown(prompt["response"]))
    return input("Analysis: ")

In [ ]:
# Load prompts from file.
with open(PROMPTS, "r") as fp:
    prompts = json.load(fp)

# Perform the zeroshot prompting.
for prompt in prompts["prompts"]:
    if "analysis" not in prompt or input("Override analysis? (y/n)").lower() in ["y", "yes"]:
        analysis = analyze(prompt)
        prompt["analysis"] = analysis

Override analysis? (y/n) n
Override analysis? (y/n) y


Write a Python TaskVine application to count the number of times the keywords "wealth", "nation", "labour", "price", "nature", and "commodity" appear in Adam Smith's Wealth of Nations. The text is available at "https://www.gutenberg.org/cache/epub/3300/pg3300.txt".


```python
import ndcctools.taskvine as vine
import requests

# Download the text of "Wealth of Nations"
url = "https://www.gutenberg.org/cache/epub/3300/pg3300.txt"
response = requests.get(url)
text = response.text

# Save the text to a file
with open("wealth_of_nations.txt", "w") as file:
    file.write(text)

# Keywords to search for
keywords = ["wealth", "nation", "labour", "price", "nature", "commodity"]

# Create a TaskVine application
app = vine.Application()

# Function to create a task for each keyword
def create_count_task(keyword):
    task = vine.Task(command=f"grep -o -i '\\b{keyword}\\b' wealth_of_nations.txt | wc -l")
    task.set_name(f"count_{keyword}")
    return task

# Add tasks to the application
for keyword in keywords:
    task = create_count_task(keyword)
    app.add_task(task)

# Run the application
app.run()

# Collect and print the results
for keyword in keywords:
    task = app.get_task_by_name(f"count_{keyword}")
    count = task.stdout.strip()
    print(f"The keyword '{keyword}' appears {count} times.")
```

Analysis:  q
Override analysis? (y/n) y


Write a Python TaskVine application to run my program "script.sh" with various input arguments and organizes all output data. "script.sh" takes, as input, an integer and two files. "script.sh" outputs a file named "output{i}.txt" where "{i}" is the input integer. The application must run "script.sh" with input integers 1-1000 and input files "file1.input" and "file2.input" which are stored in the directory "script_inputs". The application must store all output files in a directory named "script_outputs".


```python
import os
from ndcctools.taskvine import Manager, Task

# Create the output directory if it doesn't exist
output_dir = "script_outputs"
os.makedirs(output_dir, exist_ok=True)

# Initialize the TaskVine manager
manager = Manager()

# Define the input files
input_file1 = "script_inputs/file1.input"
input_file2 = "script_inputs/file2.input"

# Add tasks to the manager
for i in range(1, 1001):
    output_file = f"output{i}.txt"
    task = Task(f"./script.sh {i} {input_file1} {input_file2}")
    task.set_input_files([input_file1, input_file2])
    task.set_output_files([output_file])
    task.specify_output_file(output_file, os.path.join(output_dir, output_file))
    manager.add_task(task)

# Wait for all tasks to complete
manager.wait_for_all()
```